In [ ]:
# Found Elements Results Analysis with Enhanced Domain-Based Processing
# Improved version with expandable URL lists and better organization

import pymongo
import pandas as pd
from collections import defaultdict
from urllib.parse import urlparse
from IPython.display import HTML, display
import json
import html

# --- Configuration ---
MONGO_HOST = 'mongo'
MONGO_PORT = 27017
MONGO_USER = 'admin'
MONGO_PASS = 'changeme'
MONGO_DB_NAME = 'tasks'
COLLECTIONS_TO_ANALYZE = ['network_scraper', 'api_scraper', 'term_scraper']

# Support for multiple task names - modify this list as needed
# Empty list means analyze ALL tasks in the collection
TASK_NAMES_TO_ANALYZE = []  # Add specific task names here: ['task1', 'task2', 'task3'] or leave empty for all

def get_main_domain_from_url(url):
    """Extract main domain from URL (consistent with scrapers)"""
    try:
        parsed_url = urlparse(url)
        netloc = parsed_url.netloc
        
        subdomains_to_remove = ["sso.", "idp.", "login.", "www."]
        for subdomain in subdomains_to_remove:
            if netloc.startswith(subdomain):
                netloc = netloc[len(subdomain):]
        
        parts = netloc.split(".")
        if len(parts) > 2:
            netloc = ".".join(parts[-2:])
        
        return f"{parsed_url.scheme}://{netloc}"
    except Exception:
        return url

def load_all_data(db):
    """Load data from all collections and organize by domain with task filtering"""
    all_data = defaultdict(lambda: defaultdict(list))
    
    for collection_name in COLLECTIONS_TO_ANALYZE:
        collection = db[collection_name]
        
        # Build query for specific task names if provided
        query = {}
        if TASK_NAMES_TO_ANALYZE:
            query = {"task_name": {"$in": TASK_NAMES_TO_ANALYZE}}
        
        data = list(collection.find(query))
        
        print(f"✅ {collection_name}: {len(data)} documents for tasks: {TASK_NAMES_TO_ANALYZE}")
        
        for doc in data:
            if 'result' in doc and doc['result'] is not None:
                result = doc['result']
                if isinstance(result, dict):
                    url = result.get('url', 'unknown')
                    domain = result.get('url_id', get_main_domain_from_url(url))
                    
                    # Store the result with collection info
                    result['scraper_type'] = collection_name
                    # Ensure we capture the task_name for badges
                    if 'task_name' not in result and 'task_name' in doc:
                        result['task_name'] = doc['task_name']
                    all_data[domain][collection_name].append(result)
    
    return all_data

def analyze_domain_elements(domain_data):
    """Analyze all elements found for a domain across all scrapers"""
    elements = {
        'secure_indicators': [],
        'possible_indicators': [],
        'api_calls': [],
        'network_findings': [],
        'term_findings': [],
        'term_details': [],  # New: detailed term findings with context
        'errors': [],
        'all_urls': set(),
        'has_secure_network': False,
        'has_secure_api': False,
        'has_secure_term': False,
        'has_secure_html_elements': False,  # New: track HTML elements
        'has_possible_network': False,     # New: track possible network indicators
        'has_possible_api': False,         # New: track possible API indicators
        'has_possible_terms': False,       # New: track possible terms
        'has_possible_html_elements': False, # New: track possible HTML elements
        'scraper_types': set(),  # Track which scrapers found results for this domain
        # Neue spezifische Flags für sichere Indikatoren nach Scraper-Typ - KORRIGIERT für task-spezifische Badges
        'has_secure_api_scraper': False,      # Nur sichere API Calls von API Scraper (keine HTML Elemente)
        'has_secure_network_scraper': False,  # Nur sichere Network Requests/Responses von Network Scraper (keine HTML Elemente)
        'has_secure_term_scraper': False,     # Nur sichere Terms von Term Scraper (keine HTML Elemente)
        'has_secure_api_calls': False,        # Spezifisch für API Calls
        'has_secure_network_requests': False, # Spezifisch für Network Requests/Responses
        'has_secure_terms_found': False,      # Spezifisch für Terms Found
    }
    
    # Analyze API Scraper data
    for result in domain_data.get('api_scraper', []):
        elements['all_urls'].add(result.get('url', 'unknown'))
        elements['scraper_types'].add('api_scraper')
        
        # Get task name for badges
        task_name = result.get('task_name', 'Unknown Task')
        
        # Secure API indicators - NUR API CALLS zählen für api_scraper Badge, HTML Elemente werden ignoriert
        if result.get('api_calls_public_key') and isinstance(result['api_calls_public_key'], list):
            for call in result['api_calls_public_key']:
                elements['secure_indicators'].append({
                    'type': 'API Call (Public Key)',
                    'url': result.get('url'),
                    'details': call,
                    'scraper': 'api_scraper',
                    'task_name': task_name
                })
                elements['api_calls'].append(call)
                elements['has_secure_api'] = True
                elements['has_secure_api_calls'] = True    # Für API-spezifische Suche
                elements['has_secure_api_scraper'] = True  # Badge nur für API Calls, nicht HTML Elemente
        
        # HTML Elemente werden zu sicheren Indikatoren hinzugefügt, aber NICHT für Badge-Vergabe verwendet
        if result.get('webauthn_input_found', False):
            elements['secure_indicators'].append({
                'type': 'WebAuthn Input Field',
                'url': result.get('url'),
                'details': result.get('webauthn_input_element', {}),
                'scraper': 'api_scraper',
                'task_name': task_name
            })
            elements['has_secure_api'] = True
            elements['has_secure_html_elements'] = True  # HTML element found
            # KEIN elements['has_secure_api_scraper'] = True hier - Badge nur für API Calls!
        
        if result.get('passkey_button_found', False):
            elements['secure_indicators'].append({
                'type': 'Passkey Button',
                'url': result.get('url'),
                'details': result.get('passkey_button_element', {}),
                'scraper': 'api_scraper',
                'task_name': task_name
            })
            elements['has_secure_api'] = True
            elements['has_secure_html_elements'] = True  # HTML element found
            # KEIN elements['has_secure_api_scraper'] = True hier - Badge nur für API Calls!
        
        # FedCM indicators
        if result.get('api_calls_identity') and isinstance(result['api_calls_identity'], list):
            for call in result['api_calls_identity']:
                elements['possible_indicators'].append({
                    'type': 'FedCM Identity Call',
                    'url': result.get('url'),
                    'details': call,
                    'scraper': 'api_scraper',
                    'task_name': task_name
                })
                elements['api_calls'].append(call)
                elements['has_possible_api'] = True  # Possible API indicator
        
        # Error tracking
        if result.get('error', False):
            elements['errors'].append({
                'url': result.get('url'),
                'scraper': 'api_scraper',
                'error_messages': result.get('workflow_errors', []),
                'task_name': task_name
            })
    
    # Analyze Network Scraper data
    for result in domain_data.get('network_scraper', []):
        elements['all_urls'].add(result.get('url', 'unknown'))
        elements['scraper_types'].add('network_scraper')
        
        # Get task name for badges
        task_name = result.get('task_name', 'Unknown Task')
        
        # Secure network indicators - NUR NETWORK REQUESTS/RESPONSES zählen für network_scraper Badge
        secure_requests = result.get('secure_passkey_requests', [])
        secure_responses = result.get('secure_passkey_responses', [])
        
        if isinstance(secure_requests, list) and secure_requests:
            for request in secure_requests:
                elements['secure_indicators'].append({
                    'type': 'Secure Network Request',
                    'url': result.get('url'),
                    'details': request,
                    'scraper': 'network_scraper',
                    'task_name': task_name
                })
                elements['network_findings'].append(request)
                elements['has_secure_network'] = True
                elements['has_secure_network_requests'] = True    # Für Network-spezifische Suche
                elements['has_secure_network_scraper'] = True     # Badge für Network Requests/Responses
        
        if isinstance(secure_responses, list) and secure_responses:
            for response in secure_responses:
                elements['secure_indicators'].append({
                    'type': 'Secure Network Response',
                    'url': result.get('url'),
                    'details': response,
                    'scraper': 'network_scraper',
                    'task_name': task_name
                })
                elements['network_findings'].append(response)
                elements['has_secure_network'] = True
                elements['has_secure_network_requests'] = True    # Für Network-spezifische Suche
                elements['has_secure_network_scraper'] = True     # Badge für Network Requests/Responses
                
        # HTML Elemente werden zu sicheren Indikatoren hinzugefügt, aber NICHT für Badge-Vergabe verwendet
        if result.get('webauthn_input_found', False):
            elements['secure_indicators'].append({
                'type': 'WebAuthn Input Field',
                'url': result.get('url'),
                'details': result.get('webauthn_input_element', {}),
                'scraper': 'network_scraper',
                'task_name': task_name
            })
            elements['has_secure_network'] = True
            elements['has_secure_html_elements'] = True  # HTML element found
            # KEIN elements['has_secure_network_scraper'] = True hier - Badge nur für Network Requests/Responses!
            
        if result.get('passkey_button_found', False):
            elements['secure_indicators'].append({
                'type': 'Passkey Button',
                'url': result.get('url'),
                'details': result.get('passkey_button_element', {}),
                'scraper': 'network_scraper',
                'task_name': task_name
            })
            elements['has_secure_network'] = True
            elements['has_secure_html_elements'] = True  # HTML element found
            # KEIN elements['has_secure_network_scraper'] = True hier - Badge nur für Network Requests/Responses!
        
        # Possible network indicators
        possible_requests = result.get('possible_passkey_requests', [])
        possible_responses = result.get('possible_passkey_responses', [])
        
        if isinstance(possible_requests, list) and possible_requests:
            for request in possible_requests:
                elements['possible_indicators'].append({
                    'type': 'Possible Network Request',
                    'url': result.get('url'),
                    'details': request,
                    'scraper': 'network_scraper',
                    'task_name': task_name
                })
                elements['has_possible_network'] = True  # Possible network indicator
        
        if isinstance(possible_responses, list) and possible_responses:
            for response in possible_responses:
                elements['possible_indicators'].append({
                    'type': 'Possible Network Response',
                    'url': result.get('url'),
                    'details': response,
                    'scraper': 'network_scraper',
                    'task_name': task_name
                })
                elements['has_possible_network'] = True  # Possible network indicator
        
        # Error tracking
        if result.get('error', False):
            elements['errors'].append({
                'url': result.get('url'),
                'scraper': 'network_scraper',
                'error_messages': result.get('error_messages', []),
                'task_name': task_name
            })
    
    # Analyze Term Scraper data
    for result in domain_data.get('term_scraper', []):
        elements['all_urls'].add(result.get('url', 'unknown'))
        elements['scraper_types'].add('term_scraper')
        
        # Get task name for badges
        task_name = result.get('task_name', 'Unknown Task')
        
        # Secure term indicators - NUR TERM-BASIERTE FINDINGS zählen für term_scraper Badge
        secure_terms = {
            'navigator_credentials_get({publickey_': 'Secure Passkey API (Get)',
            'navigator_credentials_create({publickey_': 'Secure Passkey API (Create)',
            # HTML Element Terms werden NICHT für Badge-Vergabe verwendet
            # 'webauthn_input_found': 'WebAuthn Input Found',
            # 'passkey_button_found': 'Passkey Button Found'
        }
        
        # Separate HTML Element Terms - diese setzen KEINE Badges
        html_element_terms = {
            'webauthn_input_found': 'WebAuthn Input Found',
            'passkey_button_found': 'Passkey Button Found'
        }
        
        # Verarbeite zuerst die Terms, die Badges setzen (API-basierte Terms)
        for term_key, term_name in secure_terms.items():
            term_value = result.get(term_key, False)
            if term_value:
                # For hit-based terms, get the hits
                hits_key = f"{term_key}_hits"
                hits = result.get(hits_key, [])
                
                elements['secure_indicators'].append({
                    'type': f'Term: {term_name}',
                    'url': result.get('url'),
                    'details': {'term': term_key, 'hits': hits},
                    'scraper': 'term_scraper',
                    'task_name': task_name
                })
                elements['term_findings'].extend(hits if isinstance(hits, list) else [])
                elements['has_secure_term'] = True
                elements['has_secure_terms_found'] = True     # Für Term-spezifische Suche
                elements['has_secure_term_scraper'] = True    # Badge für Term-basierte Findings
                
                # Add detailed term findings for the new "Terms Found" category
                if isinstance(hits, list):
                    for hit in hits:
                        elements['term_details'].append({
                            'term_name': term_name,
                            'term_key': term_key,
                            'url': result.get('url'),
                            'task_name': task_name,
                            'hit_details': hit,
                            'scraper': 'term_scraper',
                            'category': 'secure'
                        })
        
        # Verarbeite HTML Element Terms - diese setzen KEINE Badges
        for term_key, term_name in html_element_terms.items():
            term_value = result.get(term_key, False)
            if term_value:
                # For boolean terms (webauthn_input_found, passkey_button_found), get element details
                if term_key == 'webauthn_input_found':
                    element_details = result.get('webauthn_input_element', {})
                elif term_key == 'passkey_button_found':
                    element_details = result.get('passkey_button_element', {})
                
                hits = [{'element_details': element_details}] if element_details else []
                
                elements['secure_indicators'].append({
                    'type': f'Term: {term_name}',
                    'url': result.get('url'),
                    'details': {'term': term_key, 'hits': hits},
                    'scraper': 'term_scraper',
                    'task_name': task_name
                })
                elements['term_findings'].extend(hits if isinstance(hits, list) else [])
                elements['has_secure_term'] = True
                elements['has_secure_html_elements'] = True  # HTML element found
                # KEIN elements['has_secure_term_scraper'] = True hier - Badge nur für API-basierte Terms!
                
                # Add detailed term findings for the new "Terms Found" category
                if isinstance(hits, list):
                    for hit in hits:
                        elements['term_details'].append({
                            'term_name': term_name,
                            'term_key': term_key,
                            'url': result.get('url'),
                            'task_name': task_name,
                            'hit_details': hit,
                            'scraper': 'term_scraper',
                            'category': 'secure'
                        })
        
        # ...existing code...
    
    return elements

def categorize_indicators(indicators, term_details=None):
    """Categorize indicators into HTML elements, API calls, network requests, and terms"""
    categories = {
        'html_elements': [],
        'api_calls': [],
        'network_requests': [],
        'terms_found': []  # New category for terms
    }
    
    for indicator in indicators:
        indicator_type = indicator.get('type', '').lower()
        
        # HTML Elements
        if any(term in indicator_type for term in ['webauthn input', 'passkey button', 'input field', 'button', 'element']):
            categories['html_elements'].append(indicator)
        # API Calls  
        elif any(term in indicator_type for term in ['api call', 'fedcm', 'identity call', 'credentials']) and 'term:' not in indicator_type:
            categories['api_calls'].append(indicator)
        # Network Requests/Responses
        elif any(term in indicator_type for term in ['network request', 'network response', 'request', 'response']):
            categories['network_requests'].append(indicator)
        # Terms - moved to separate category
        elif 'term:' in indicator_type:
            categories['terms_found'].append(indicator)
        else:
            # Default category for unmatched indicators
            categories['api_calls'].append(indicator)
    
    return categories

def create_terms_found_html(term_details, category_type):
    """Create HTML for the new Terms Found category"""
    if not term_details:
        return ""
    
    # Filter terms by category (secure or possible)
    filtered_terms = [term for term in term_details if term.get('category') == category_type]
    
    if not filtered_terms:
        return ""
    
    border_color = "#28a745" if category_type == "secure" else "#e67e22"
    
    html_content = f'''
    <details style="margin: 15px 0; border: 1px solid {border_color}; border-radius: 8px;">
        <summary style="font-weight: bold; color: {border_color}; cursor: pointer; padding: 15px; background: rgba(255,255,255,0.5); border-radius: 8px;">
            📝 Terms Found ({len(filtered_terms)})
        </summary>
        <div style="padding: 15px;">
    '''
    
    for term in filtered_terms:
        hit_details = term.get('hit_details', {})
        
        # Get the actual search term that was found
        term_key = term.get('term_key', 'N/A')
        
        # Add task name badge
        task_name = term.get('task_name', 'Unknown Task')
        badge_class = "term"
        task_badge = f'<span class="task-badge {badge_class}" title="{task_name}">{task_name}</span>'
        
        html_content += f'''
            <div style="border-left: 3px solid {border_color}; padding-left: 15px; margin: 12px 0; background: white; border-radius: 8px; padding: 12px;">
                <strong>Term:</strong> {term.get('term_name', 'Unknown')} {task_badge}<br>
                <strong>URL:</strong> <a href="{term.get('url', '#')}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px;">{term.get('url', 'N/A')}</a><br>
                <strong>Search Term Found:</strong> <span style="font-family: monospace; background: #f8f9fa; padding: 2px 4px; border-radius: 3px; color: #007bff; font-weight: bold;">{term_key}</span><br>
                <details style="margin-top: 10px;">
                    <summary style="cursor: pointer; color: #666; font-weight: bold;">Show Full Details</summary>
                    <pre style="background: #f8f9fa; padding: 12px; border-radius: 6px; font-size: 12px; max-height: 250px; overflow-y: auto; margin: 8px 0; border: 1px solid #e9ecef;">{html.escape(json.dumps(hit_details, indent=2))}</pre>
                </details>
            </div>
        '''
    
    html_content += '''
        </div>
    </details>
    '''
    
    return html_content

def create_domain_analysis_html(domain_data):
    """Create HTML output for domain-based analysis with expandable URL lists"""
    html_content = []
    
    # Add debugging information at the start
    total_domains = len(domain_data)
    print(f"🔍 DEBUG: Total domains to process: {total_domains}")
    
    # Add modern styling and search functionality
    html_content.append('''
    <style>
    .search-container {
        background: linear-gradient(135deg, #74b9ff 0%, #0984e3 100%);
        padding: 20px;
        border-radius: 15px;
        margin: 20px 0;
        box-shadow: 0 8px 25px rgba(0,0,0,0.15);
    }
    .search-input {
        width: 100%;
        padding: 12px 20px;
        border: none;
        border-radius: 25px;
        font-size: 16px;
        box-shadow: 0 4px 10px rgba(0,0,0,0.1);
        outline: none;
        transition: box-shadow 0.3s ease;
    }
    .search-input:focus {
        box-shadow: 0 6px 20px rgba(0,0,0,0.2);
    }
    .domain-item {
        transition: all 0.3s ease;
        margin: 15px 0;
    }
    .domain-item.hidden {
        display: none;
    }
    .clickable-url {
        color: #0066cc;
        text-decoration: none;
        transition: color 0.3s ease;
    }
    .clickable-url:hover {
        color: #004499;
        text-decoration: underline;
    }
    .indicator-category {
        background: #f8f9fa;
        border-radius: 10px;
        margin: 10px 0;
        border-left: 4px solid #dee2e6;
    }
    .indicator-category.html-elements {
        border-left-color: #28a745;
    }
    .indicator-category.api-calls {
        border-left-color: #007bff;
    }
    .indicator-category.network-requests {
        border-left-color: #ffc107;
    }
    .task-badge {
        display: inline-block;
        padding: 3px 8px;
        border-radius: 12px;
        font-size: 11px;
        font-weight: 600;
        background: #e9ecef;
        color: #495057;
        margin-left: 8px;
        white-space: nowrap;
        overflow: hidden;
        text-overflow: ellipsis;
        max-width: 200px;
        vertical-align: middle;
        border: 1px solid #ced4da;
    }
    .task-badge.api {
        background: #cce5ff;
        color: #004085;
        border-color: #b8daff;
    }
    .task-badge.network {
        background: #fff3cd;
        color: #856404;
        border-color: #ffeeba;
    }
    .task-badge.term {
        background: #d4edda;
        color: #155724;
        border-color: #c3e6cb;
    }
    </style>
    ''')

    # Initialize counters for different categories - KORRIGIERT
    category_counts = {
        'passkey': 0,
        'network': 0,
        'api': 0,
        'terms': 0,
        'html_elements': 0,  # Neu hinzugefügt
        'possible_network': 0,  # Neu hinzugefügt
        'possible_api': 0,  # Neu hinzugefügt
        'possible_terms': 0,  # Neu hinzugefügt
        'network_scraper': 0,
        'api_scraper': 0,
        'term_scraper': 0
    }
    
    domains_with_elements = 0
    domains_without_elements = 0
    total_secure_indicators = 0
    total_possible_indicators = 0
    domain_counter = 0
    
    # First pass to count domains in each category - KORRIGIERT
    for domain, data in domain_data.items():
        elements = analyze_domain_elements(data)
        
        has_secure_indicators = len(elements['secure_indicators']) > 0
        has_possible_indicators = len(elements['possible_indicators']) > 0
        has_any_indicators = has_secure_indicators or has_possible_indicators
        
        # Korrekte Zählung der Indikatoren
        total_secure_indicators += len(elements['secure_indicators'])
        total_possible_indicators += len(elements['possible_indicators'])
        
        if has_any_indicators:
            domains_with_elements += 1
        else:
            domains_without_elements += 1
        
        # Count by secure category - nur wenn tatsächlich sichere Indikatoren vorhanden sind
        if has_secure_indicators:
            category_counts['passkey'] += 1
            
            # Spezifische Zählung nach Indikator-Typ - KORRIGIERT
            if elements['has_secure_network_requests']:  # Nur Network Requests/Responses
                category_counts['network'] += 1
                
            if elements['has_secure_api_calls']:         # Nur API Calls
                category_counts['api'] += 1
                
            if elements['has_secure_terms_found']:       # Nur Terms Found
                category_counts['terms'] += 1
                
            if elements['has_secure_html_elements']:
                category_counts['html_elements'] += 1
        
        # Count by possible category - nur wenn tatsächlich mögliche Indikatoren vorhanden sind
        if has_possible_indicators:
            if elements['has_possible_network']:
                category_counts['possible_network'] += 1
                
            if elements['has_possible_api']:
                category_counts['possible_api'] += 1
                
            if elements['has_possible_terms']:
                category_counts['possible_terms'] += 1
        
        # Count by scraper type - nur wenn der Scraper tatsächlich Ergebnisse gefunden hat
        for scraper_type in elements['scraper_types']:
            if scraper_type in category_counts:
                category_counts[scraper_type] += 1
    
    # Updated search badges with correct counts - KORRIGIERT
    search_badges = f'''
    <div style="display: flex; flex-wrap: wrap; margin-top: 15px; gap: 10px;">
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white;">
            <strong>passkey:</strong> {category_counts['passkey']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white;">
            <strong>network:</strong> {category_counts['network']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white;">
            <strong>api:</strong> {category_counts['api']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white;">
            <strong>terms:</strong> {category_counts['terms']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white;">
            <strong>html_elements:</strong> {category_counts['html_elements']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white; border-left: 3px solid rgba(255,193,7,0.7);">
            <strong>possible_network:</strong> {category_counts['possible_network']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white; border-left: 3px solid rgba(255,193,7,0.7);">
            <strong>possible_api:</strong> {category_counts['possible_api']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white; border-left: 3px solid rgba(255,193,7,0.7);">
            <strong>possible_terms:</strong> {category_counts['possible_terms']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white; border-left: 3px solid rgba(255,255,255,0.5);">
            <strong>network_scraper:</strong> {category_counts['network_scraper']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white; border-left: 3px solid rgba(255,255,255,0.5);">
            <strong>api_scraper:</strong> {category_counts['api_scraper']} Domains
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 8px 12px; border-radius: 20px; font-size: 14px; color: white; border-left: 3px solid rgba(255,255,255,0.5);">
            <strong>term_scraper:</strong> {category_counts['term_scraper']} Domains
        </div>
    </div>
    <div style="margin-top: 10px; font-size: 12px; color: rgba(255,255,255,0.8);">
        <strong>Secure filters:</strong> 'passkey', 'network', 'api', 'terms', 'html_elements'<br>
        <strong>Possible indicators:</strong> 'possible_network', 'possible_api', 'possible_terms'<br>
        <strong>By scraper:</strong> 'network_scraper', 'api_scraper', 'term_scraper'
    </div>
    '''
    
    html_content.append('''
    <h2 style="background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%); 
               background-clip: text; -webkit-background-clip: text; color: transparent; 
               font-size: 28px; margin: 30px 0 20px 0; font-weight: bold;">
        🔍 Domain-Based Element Analysis
    </h2>
    ''')
    
    # Create the search container with updated help text
    search_container_html = f'''
    <div class="search-container">
        <h4 style="color: white; margin: 0 0 15px 0;">🔎 Search Domains</h4>
        <input type="text" class="search-input" id="domainSearch" 
               placeholder="Enter search term (e.g., 'google', 'login', 'auth')..." 
               onkeyup="filterDomains()">
        <p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 14px;">
            <strong>Secure indicators:</strong> 'passkey' (all secure), 'network', 'api', 'terms', 'html_elements'<br>
            <strong>Possible indicators:</strong> 'possible_network', 'possible_api', 'possible_terms'<br>
            <strong>By scraper:</strong> 'network_scraper', 'api_scraper', 'term_scraper'<br>
            <strong>Total Domains Found: {total_domains}</strong>
        </p>
        
        {search_badges}
    </div>
    '''
    
    # Updated JavaScript with corrected filter logic - KORRIGIERT
    javascript_code = '''
    <script>
    function filterDomains() {
        const searchTerm = document.getElementById('domainSearch').value.toLowerCase().trim();
        const domainItems = document.querySelectorAll('.domain-item');
        let visibleCount = 0;
        
        domainItems.forEach(item => {
            const domainText = item.getAttribute('data-domain').toLowerCase();
            const urlTexts = item.getAttribute('data-urls').toLowerCase();
            const hasSecureNetwork = item.getAttribute('data-secure-network') === 'true';
            const hasSecureApi = item.getAttribute('data-secure-api') === 'true';
            const hasSecureTerm = item.getAttribute('data-secure-term') === 'true';
            const hasSecureHtmlElements = item.getAttribute('data-secure-html-elements') === 'true';
            const hasPossibleNetwork = item.getAttribute('data-possible-network') === 'true';
            const hasPossibleApi = item.getAttribute('data-possible-api') === 'true';
            const hasPossibleTerms = item.getAttribute('data-possible-terms') === 'true';
            const hasPossibleHtmlElements = item.getAttribute('data-possible-html-elements') === 'true';
            const scraperTypes = item.getAttribute('data-scraper-types').toLowerCase();
            const hasSecureIndicator = item.getAttribute('data-has-secure-indicator') === 'true';
            
            // Neue spezifische Attribute für präzise Filterung - HINZUGEFÜGT
            const hasSecureApiCalls = item.getAttribute('data-secure-api-calls') === 'true';
            const hasSecureNetworkRequests = item.getAttribute('data-secure-network-requests') === 'true';
            const hasSecureTermsFound = item.getAttribute('data-secure-terms-found') === 'true';
            
            let showItem = false;
            
            if (searchTerm === '' || searchTerm.length === 0) {
                showItem = true; // Show all when search is empty
            } else {
                // Exact filter matches - nur Domains mit den spezifischen Eigenschaften
                switch (searchTerm) {
                    case 'passkey':
                        showItem = hasSecureIndicator;
                        break;
                    case 'network':
                        // Nur Domains mit sicheren Network Requests/Responses - KORRIGIERT
                        showItem = hasSecureNetworkRequests;
                        break;
                    case 'api':
                        // Nur Domains mit sicheren API Calls - KORRIGIERT
                        showItem = hasSecureApiCalls;
                        break;
                    case 'terms':
                        // Nur Domains mit sicheren Terms Found - KORRIGIERT
                        showItem = hasSecureTermsFound;
                        break;
                    case 'html_elements':
                        showItem = hasSecureHtmlElements;
                        break;
                    case 'possible_network':
                        showItem = hasPossibleNetwork;
                        break;
                    case 'possible_api':
                        showItem = hasPossibleApi;
                        break;
                    case 'possible_terms':
                        showItem = hasPossibleTerms;
                        break;
                    case 'network_scraper':
                        showItem = scraperTypes.includes('network_scraper');
                        break;
                    case 'api_scraper':
                        showItem = scraperTypes.includes('api_scraper');
                        break;
                    case 'term_scraper':
                        showItem = scraperTypes.includes('term_scraper');
                        break;
                    default:
                        // For non-special terms, search in domain name and URLs
                        showItem = domainText.includes(searchTerm) || urlTexts.includes(searchTerm);
                        break;
                }
            }
            
            if (showItem) {
                item.classList.remove('hidden');
                visibleCount++;
            } else {
                item.classList.add('hidden');
            }
        });
        
        // Update search result count
        const resultInfo = document.getElementById('searchResults');
        if (resultInfo) {
            if (searchTerm === '') {
                resultInfo.textContent = `${visibleCount} von ''' + str(total_domains) + ''' Domains angezeigt`;
            } else {
                resultInfo.textContent = `${visibleCount} von ''' + str(total_domains) + ''' Domains gefunden für "${searchTerm}"`;
            }
        }
    }
    
    // Initialize display on page load
    document.addEventListener('DOMContentLoaded', function() {
        filterDomains(); // Show all domains initially
    });
    </script>
    '''
    
    html_content.append(search_container_html)
    html_content.append(javascript_code)
    
    html_content.append(f'''
    <div id="searchResults" style="margin: 10px 0; font-weight: bold; color: #666;">
        {total_domains} von {total_domains} Domains angezeigt
    </div>
    ''')
    
    html_content.append('<div style="margin: 20px 0;"><em>Click on a domain to show all details.</em></div>')
    
    # Reset domain counter for the actual display loop - KORRIGIERT
    domain_counter = 0
    
    # Sort domains alphabetically for consistent display
    sorted_domains = sorted(domain_data.items())
    
    for domain, data in sorted_domains:
        elements = analyze_domain_elements(data)
        
        # Always increment domain counter to show all domains
        domain_counter += 1
        
        has_indicators = len(elements['secure_indicators']) > 0 or len(elements['possible_indicators']) > 0
        
        # Convert scraper_types set to string for data attribute
        scraper_types_str = ' '.join(elements['scraper_types'])
        
        if has_indicators:
            # Create expandable section for this domain
            secure_count = len(elements['secure_indicators'])
            possible_count = len(elements['possible_indicators'])
            error_count = len(elements['errors'])
            url_count = len(elements['all_urls']);
            
            status_color = 'green' if secure_count > 0 else 'orange';
            status_icon = '🔒' if secure_count > 0 else '⚠️';
            
            # Create data attributes for search functionality with corrected attributes - KORRIGIERT
            all_urls_str = ' '.join(elements['all_urls']);
            
            # Erstelle Task-spezifische Badges für sichere Indikatoren - KORRIGIERT mit getauschten Farben
            task_badges = []
            if elements['has_secure_api_scraper']:
                # Farbe getauscht: jetzt cyan/teal für api_scraper
                task_badges.append('<span class="task-badge api" style="background: #17a2b8; color: white; border-color: #138496;">api_scraper</span>')
            if elements['has_secure_network_scraper']:
                # Farbe bleibt gelb für network_scraper
                task_badges.append('<span class="task-badge network" style="background: #ffc107; color: #212529; border-color: #e0a800;">network_scraper</span>')
            if elements['has_secure_term_scraper']:
                # Farbe getauscht: jetzt grün für term_scraper
                task_badges.append('<span class="task-badge term" style="background: #28a745; color: white; border-color: #1e7e34;">term_scraper</span>')
            
            task_badges_html = ' '.join(task_badges) if task_badges else ''
            
            html_content.append(f'''
            <details class="domain-item" data-domain="{domain}" data-urls="{all_urls_str}" 
                     data-secure-network="{str(elements['has_secure_network']).lower()}" 
                     data-secure-api="{str(elements['has_secure_api']).lower()}" 
                     data-secure-term="{str(elements['has_secure_term']).lower()}"
                     data-secure-html-elements="{str(elements['has_secure_html_elements']).lower()}"
                     data-possible-network="{str(elements['has_possible_network']).lower()}"
                     data-possible-api="{str(elements['has_possible_api']).lower()}"
                     data-possible-terms="{str(elements['has_possible_terms']).lower()}"
                     data-possible-html-elements="{str(elements['has_possible_html_elements']).lower()}"
                     data-secure-api-calls="{str(elements['has_secure_api_calls']).lower()}"
                     data-secure-network-requests="{str(elements['has_secure_network_requests']).lower()}"
                     data-secure-terms-found="{str(elements['has_secure_terms_found']).lower()}"
                     data-scraper-types="{scraper_types_str}"
                     data-has-secure-indicator="{str(secure_count > 0).lower()}"
                     style="margin: 20px 0; border: 2px solid {status_color}; border-radius: 15px; padding: 15px; 
                            background: linear-gradient(135deg, rgba(255,255,255,0.9) 0%, rgba(248,249,250,0.9) 100%);
                            box-shadow: 0 4px 15px rgba(0,0,0,0.1); transition: all 0.3s ease;">
                <summary style="font-weight: bold; color: {status_color}; cursor: pointer; font-size: 18px; 
                               padding: 10px 0;">
                    {status_icon} {domain_counter}. {domain} - {url_count} URLs, {secure_count} secure + {possible_count} possible indicators {task_badges_html}
                </summary>
                <div style="margin: 15px 0;">
                    <details style="margin: 10px 0; background: white; border-radius: 10px; padding: 10px;">
                        <summary style="font-weight: bold; cursor: pointer; color: #2c5282; font-size: 16px;">
                            📋 URLs in this Domain ({url_count} URLs show/hide)
                        </summary>
                        <div style="margin: 15px 0; max-height: 300px; overflow-y: auto; background: #f8f9fa; padding: 15px; border-radius: 8px;">
                            <ul style="margin: 0; padding-left: 20px; list-style-type: none;">
            ''')
            
            for url in sorted(elements['all_urls']):
                html_content.append(f'''
                <li style="margin: 8px 0; padding: 5px; background: white; border-radius: 5px; border-left: 3px solid #007bff;">
                    <a href="{url}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px; text-decoration: none;">
                        🔗 {url}
                    </a>
                </li>
                ''')
            
            html_content.append('''
                            </ul>
                        </div>
                    </details>
            ''')
            
            # Secure indicators - categorized with collapsible subcategories
            if elements['secure_indicators']:
                secure_categories = categorize_indicators(elements['secure_indicators'])
                
                html_content.append(f'''
                <details style="margin: 15px 0; background: rgba(40, 167, 69, 0.05); border-radius: 10px; border: 1px solid #28a745;">
                    <summary style="font-weight: bold; color: #28a745; cursor: pointer; padding: 15px; font-size: 16px;">
                        🔒 Secure Indicators ({len(elements['secure_indicators'])})
                    </summary>
                    <div style="padding: 0 15px 15px 15px;">
                ''')
                
                # HTML Elements - Now collapsible by default
                if secure_categories['html_elements']:
                    html_content.append(f'''
                    <details style="margin: 15px 0; border: 1px solid #28a745; border-radius: 8px;">
                        <summary style="font-weight: bold; color: #28a745; cursor: pointer; padding: 15px; background: rgba(255,255,255,0.5); border-radius: 8px;">
                            🏷️ HTML Elements ({len(secure_categories['html_elements'])})
                        </summary>
                        <div style="padding: 15px;">
                    ''')
                    for indicator in secure_categories['html_elements']:
                        # Enhanced HTML element details display
                        details = indicator['details']
                        if isinstance(details, dict):
                            # For HTML elements, show HTML code prominently
                            html_code = details.get('outerHTML', details.get('innerHTML', ''))
                            if not html_code:
                                html_code = details.get('element_html', '')
                            
                            if html_code:
                                details_str = f"HTML Code:\n{html_code}\n\nComplete Details:\n{json.dumps(details, indent=2)}"
                            else:
                                details_str = json.dumps(details, indent=2)
                        else:
                            details_str = str(details)
                        
                        # Escape HTML in details to prevent rendering as actual HTML elements
                        import html
                        details_str = html.escape(details_str)
                        
                        # Add task name badge
                        task_name = indicator.get('task_name', 'Unknown Task')
                        badge_class = ""
                        if "api" in indicator['scraper'].lower():
                            badge_class = "api"
                        elif "network" in indicator['scraper'].lower():
                            badge_class = "network"
                        elif "term" in indicator['scraper'].lower():
                            badge_class = "term"
                        
                        task_badge = f'<span class="task-badge {badge_class}" title="{task_name}">{task_name}</span>'
                        
                        html_content.append(f'''
                            <div style="border-left: 3px solid #28a745; padding-left: 15px; margin: 12px 0; background: white; border-radius: 8px; padding: 12px;">
                                <strong>Type:</strong> {indicator['type']} {task_badge}<br>
                                <strong>URL:</strong> <a href="{indicator['url']}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px;">{indicator['url']}</a><br>
                                <strong>Scraper:</strong> {indicator['scraper']}<br>
                                <details style="margin-top: 10px;">
                                    <summary style="cursor: pointer; color: #666; font-weight: bold;">Show Details</summary>
                                    <pre style="background: #f8f9fa; padding: 12px; border-radius: 6px; font-size: 12px; max-height: 250px; overflow-y: auto; margin: 8px 0; border: 1px solid #e9ecef;">{details_str}</pre>
                                </details>
                            </div>
                        ''')
                    html_content.append('</div></details>')
                
                # API Calls - Now collapsible by default
                if secure_categories['api_calls']:
                    html_content.append(f'''
                    <details style="margin: 15px 0; border: 1px solid #28a745; border-radius: 8px;">
                        <summary style="font-weight: bold; color: #28a745; cursor: pointer; padding: 15px; background: rgba(255,255,255,0.5); border-radius: 8px;">
                            🔧 API Calls ({len(secure_categories['api_calls'])})
                        </summary>
                        <div style="padding: 15px;">
                    ''')
                    for indicator in secure_categories['api_calls']:
                        details_str = json.dumps(indicator['details'], indent=2) if isinstance(indicator['details'], dict) else str(indicator['details'])
                        import html
                        details_str = html.escape(details_str)
                        
                        # Add task name badge
                        task_name = indicator.get('task_name', 'Unknown Task')
                        badge_class = ""
                        if "api" in indicator['scraper'].lower():
                            badge_class = "api"
                        elif "network" in indicator['scraper'].lower():
                            badge_class = "network"
                        elif "term" in indicator['scraper'].lower():
                            badge_class = "term"
                        
                        task_badge = f'<span class="task-badge {badge_class}" title="{task_name}">{task_name}</span>'
                        
                        html_content.append(f'''
                            <div style="border-left: 3px solid #007bff; padding-left: 15px; margin: 12px 0; background: white; border-radius: 8px; padding: 12px;">
                                <strong>Type:</strong> {indicator['type']} {task_badge}<br>
                                <strong>URL:</strong> <a href="{indicator['url']}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px;">{indicator['url']}</a><br>
                                <strong>Scraper:</strong> {indicator['scraper']}<br>
                                <details style="margin-top: 10px;">
                                    <summary style="cursor: pointer; color: #666; font-weight: bold;">Show Details</summary>
                                    <pre style="background: #f8f9fa; padding: 12px; border-radius: 6px; font-size: 12px; max-height: 250px; overflow-y: auto; margin: 8px 0; border: 1px solid #e9ecef;">{details_str}</pre>
                                </details>
                            </div>
                        ''')
                    html_content.append('</div></details>')
                
                # Network Requests/Responses - Now collapsible by default
                if secure_categories['network_requests']:
                    html_content.append(f'''
                    <details style="margin: 15px 0; border: 1px solid #28a745; border-radius: 8px;">
                        <summary style="font-weight: bold; color: #28a745; cursor: pointer; padding: 15px; background: rgba(255,255,255,0.5); border-radius: 8px;">
                            🌐 Network Requests/Responses ({len(secure_categories['network_requests'])})
                        </summary>
                        <div style="padding: 15px;">
                    ''')
                    for indicator in secure_categories['network_requests']:
                        details_str = json.dumps(indicator['details'], indent=2) if isinstance(indicator['details'], dict) else str(indicator['details'])
                        import html
                        details_str = html.escape(details_str)
                        
                        # Add task name badge
                        task_name = indicator.get('task_name', 'Unknown Task')
                        badge_class = ""
                        if "api" in indicator['scraper'].lower():
                            badge_class = "api"
                        elif "network" in indicator['scraper'].lower():
                            badge_class = "network"
                        elif "term" in indicator['scraper'].lower():
                            badge_class = "term"
                        
                        task_badge = f'<span class="task-badge {badge_class}" title="{task_name}">{task_name}</span>'
                        
                        html_content.append(f'''
                            <div style="border-left: 3px solid #ffc107; padding-left: 15px; margin: 12px 0; background: white; border-radius: 8px; padding: 12px;">
                                <strong>Type:</strong> {indicator['type']} {task_badge}<br>
                                <strong>URL:</strong> <a href="{indicator['url']}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px;">{indicator['url']}</a><br>
                                <strong>Scraper:</strong> {indicator['scraper']}<br>
                                <details style="margin-top: 10px;">
                                    <summary style="cursor: pointer; color: #666; font-weight: bold;">Show Details</summary>
                                    <pre style="background: #f8f9fa; padding: 12px; border-radius: 6px; font-size: 12px; max-height: 250px; overflow-y: auto; margin: 8px 0; border: 1px solid #e9ecef;">{details_str}</pre>
                                </details>
                            </div>
                        ''')
                    html_content.append('</div></details>')
                
                # Terms Found - New collapsible category
                terms_html = create_terms_found_html(elements['term_details'], 'secure')
                if terms_html:
                    html_content.append(terms_html)
                
                html_content.append('</div></details>')
            
            # Possible indicators - categorized with collapsible subcategories
            if elements['possible_indicators']:
                possible_categories = categorize_indicators(elements['possible_indicators'])
                
                html_content.append(f'''
                <details style="margin: 15px 0; background: rgba(255, 193, 7, 0.05); border-radius: 10px; border: 1px solid #ffc107;">
                    <summary style="font-weight: bold; color: #e67e22; cursor: pointer; padding: 15px; font-size: 16px;">
                        ⚠️ Possible Indicators ({len(elements['possible_indicators'])})
                    </summary>
                    <div style="padding: 0 15px 15px 15px;">
                ''')
                
                # HTML Elements - Now collapsible by default
                if possible_categories['html_elements']:
                    html_content.append(f'''
                    <details style="margin: 15px 0; border: 1px solid #e67e22; border-radius: 8px;">
                        <summary style="font-weight: bold; color: #e67e22; cursor: pointer; padding: 15px; background: rgba(255,255,255,0.5); border-radius: 8px;">
                            🏷️ HTML Elements ({len(possible_categories['html_elements'])})
                        </summary>
                        <div style="padding: 15px;">
                    ''')
                    for indicator in possible_categories['html_elements']:
                        # Enhanced HTML element details display
                        details = indicator['details']
                        if isinstance(details, dict):
                            # For HTML elements, show HTML code prominently
                            html_code = details.get('outerHTML', details.get('innerHTML', ''))
                            if not html_code:
                                html_code = details.get('element_html', '')
                            
                            if html_code:
                                details_str = f"HTML Code:\n{html_code}\n\nComplete Details:\n{json.dumps(details, indent=2)}"
                            else:
                                details_str = json.dumps(details, indent=2)
                        else:
                            details_str = str(details)
                        
                        # Escape HTML in details to prevent rendering as actual HTML elements
                        import html
                        details_str = html.escape(details_str)
                        
                        # Add task name badge
                        task_name = indicator.get('task_name', 'Unknown Task')
                        badge_class = ""
                        if "api" in indicator['scraper'].lower():
                            badge_class = "api"
                        elif "network" in indicator['scraper'].lower():
                            badge_class = "network"
                        elif "term" in indicator['scraper'].lower():
                            badge_class = "term"
                        
                        task_badge = f'<span class="task-badge {badge_class}" title="{task_name}">{task_name}</span>'
                        
                        html_content.append(f'''
                            <div style="border-left: 3px solid #e67e22; padding-left: 15px; margin: 12px 0; background: white; border-radius: 8px; padding: 12px;">
                                <strong>Type:</strong> {indicator['type']} {task_badge}<br>
                                <strong>URL:</strong> <a href="{indicator['url']}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px;">{indicator['url']}</a><br>
                                <strong>Scraper:</strong> {indicator['scraper']}<br>
                                <details style="margin-top: 10px;">
                                    <summary style="cursor: pointer; color: #666; font-weight: bold;">Show Details</summary>
                                    <pre style="background: #f8f9fa; padding: 12px; border-radius: 6px; font-size: 12px; max-height: 250px; overflow-y: auto; margin: 8px 0; border: 1px solid #e9ecef;">{details_str}</pre>
                                </details>
                            </div>
                        ''')
                    html_content.append('</div></details>')
                
                # API Calls - Now collapsible by default
                if possible_categories['api_calls']:
                    html_content.append(f'''
                    <details style="margin: 15px 0; border: 1px solid #e67e22; border-radius: 8px;">
                        <summary style="font-weight: bold; color: #e67e22; cursor: pointer; padding: 15px; background: rgba(255,255,255,0.5); border-radius: 8px;">
                            🔧 API Calls ({len(possible_categories['api_calls'])})
                        </summary>
                        <div style="padding: 15px;">
                    ''')
                    for indicator in possible_categories['api_calls']:
                        details_str = json.dumps(indicator['details'], indent=2) if isinstance(indicator['details'], dict) else str(indicator['details'])
                        import html
                        details_str = html.escape(details_str)
                        
                        # Add task name badge
                        task_name = indicator.get('task_name', 'Unknown Task')
                        badge_class = ""
                        if "api" in indicator['scraper'].lower():
                            badge_class = "api"
                        elif "network" in indicator['scraper'].lower():
                            badge_class = "network"
                        elif "term" in indicator['scraper'].lower():
                            badge_class = "term"
                        
                        task_badge = f'<span class="task-badge {badge_class}" title="{task_name}">{task_name}</span>'
                        
                        html_content.append(f'''
                            <div style="border-left: 3px solid #e67e22; padding-left: 15px; margin: 12px 0; background: white; border-radius: 8px; padding: 12px;">
                                <strong>Type:</strong> {indicator['type']} {task_badge}<br>
                                <strong>URL:</strong> <a href="{indicator['url']}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px;">{indicator['url']}</a><br>
                                <strong>Scraper:</strong> {indicator['scraper']}<br>
                                <details style="margin-top: 10px;">
                                    <summary style="cursor: pointer; color: #666; font-weight: bold;">Show Details</summary>
                                    <pre style="background: #f8f9fa; padding: 12px; border-radius: 6px; font-size: 12px; max-height: 250px; overflow-y: auto; margin: 8px 0; border: 1px solid #e9ecef;">{details_str}</pre>
                                </details>
                            </div>
                        ''')
                    html_content.append('</div></details>')
                
                # Network Requests/Responses - Now collapsible by default
                if possible_categories['network_requests']:
                    html_content.append(f'''
                    <details style="margin: 15px 0; border: 1px solid #e67e22; border-radius: 8px;">
                        <summary style="font-weight: bold; color: #e67e22; cursor: pointer; padding: 15px; background: rgba(255,255,255,0.5); border-radius: 8px;">
                            🌐 Network Requests/Responses ({len(possible_categories['network_requests'])})
                        </summary>
                        <div style="padding: 15px;">
                    ''')
                    for indicator in possible_categories['network_requests']:
                        details_str = json.dumps(indicator['details'], indent=2) if isinstance(indicator['details'], dict) else str(indicator['details'])
                        import html
                        details_str = html.escape(details_str)
                        
                        # Add task name badge
                        task_name = indicator.get('task_name', 'Unknown Task')
                        badge_class = ""
                        if "api" in indicator['scraper'].lower():
                            badge_class = "api"
                        elif "network" in indicator['scraper'].lower():
                            badge_class = "network"
                        elif "term" in indicator['scraper'].lower():
                            badge_class = "term"
                        
                        task_badge = f'<span class="task-badge {badge_class}" title="{task_name}">{task_name}</span>'
                        
                        html_content.append(f'''
                            <div style="border-left: 3px solid #e67e22; padding-left: 15px; margin: 12px 0; background: white; border-radius: 8px; padding: 12px;">
                                <strong>Type:</strong> {indicator['type']} {task_badge}<br>
                                <strong>URL:</strong> <a href="{indicator['url']}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px;">{indicator['url']}</a><br>
                                <strong>Scraper:</strong> {indicator['scraper']}<br>
                                <details style="margin-top: 10px;">
                                    <summary style="cursor: pointer; color: #666; font-weight: bold;">Show Details</summary>
                                    <pre style="background: #f8f9fa; padding: 12px; border-radius: 6px; font-size: 12px; max-height: 250px; overflow-y: auto; margin: 8px 0; border: 1px solid #e9ecef;">{details_str}</pre>
                                </details>
                            </div>
                        ''')
                    html_content.append('</div></details>')
                
                # Terms Found - New collapsible category
                terms_html = create_terms_found_html(elements['term_details'], 'possible')
                if terms_html:
                    html_content.append(terms_html)
                
                html_content.append('</div></details>')
            
            # Errors
            if elements['errors']:
                html_content.append(f'''
                <details style="margin: 15px 0; background: rgba(220, 53, 69, 0.05); border-radius: 10px; border: 1px solid #dc3545;">
                    <summary style="font-weight: bold; color: #dc3545; cursor: pointer; padding: 15px; font-size: 16px;">
                        ❌ Errors ({len(elements['errors'])})
                    </summary>
                    <div style="padding: 0 15px 15px 15px;">
                ''')
                
                for error in elements['errors']:
                    # Add task name badge for errors too
                    task_name = error.get('task_name', 'Unknown Task')
                    badge_class = ""
                    if "api" in error['scraper'].lower():
                        badge_class = "api"
                    elif "network" in error['scraper'].lower():
                        badge_class = "network"
                    elif "term" in error['scraper'].lower():
                        badge_class = "term"
                    
                    task_badge = f'<span class="task-badge {badge_class}" title="{task_name}">{task_name}</span>'
                    
                    html_content.append(f'''
                        <div style="border-left: 3px solid #dc3545; padding-left: 15px; margin: 12px 0; background: white; border-radius: 8px; padding: 12px;">
                            <strong>URL:</strong> <a href="{error['url']}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px;">{error['url']}</a><br>
                            <strong>Scraper:</strong> {error['scraper']} {task_badge}<br>
                            <strong>Errors:</strong> <span style="color: #dc3545; font-weight: bold;">{error['error_messages']}</span>
                        </div>
                    ''')
                
                html_content.append('</div></details>')
            
            html_content.append('</div></details>')
        else:
            domains_without_elements += 1
            
            # Show domains without indicators with error information
            url_count = len(elements['all_urls'])
            error_count = len(elements['errors'])
            
            status_color = '#6c757d'  # Gray for no indicators
            status_icon = '❌'
            
            # Create data attributes for search functionality - KORRIGIERT
            all_urls_str = ' '.join(elements['all_urls'])
            
            html_content.append(f'''
            <details class="domain-item" data-domain="{domain}" data-urls="{all_urls_str}" 
                     data-secure-network="false" data-secure-api="false" data-secure-term="false"
                     data-secure-html-elements="false"
                     data-possible-network="false" data-possible-api="false" data-possible-terms="false"
                     data-possible-html-elements="false"
                     data-secure-api-calls="false"
                     data-secure-network-requests="false"
                     data-secure-terms-found="false"
                     data-scraper-types="{scraper_types_str}"
                     data-has-secure-indicator="false"
                     style="margin: 20px 0; border: 2px solid {status_color}; border-radius: 15px; padding: 15px; 
                            background: linear-gradient(135deg, rgba(248,249,250,0.9) 0%, rgba(233,236,239,0.9) 100%);
                            box-shadow: 0 4px 15px rgba(0,0,0,0.1); transition: all 0.3s ease;">
                <summary style="font-weight: bold; color: {status_color}; cursor: pointer; font-size: 18px; 
                               padding: 10px 0;">
                    {status_icon} {domain_counter}. {domain} - {url_count} URLs, no indicators found
                    {' (' + ', '.join(elements['scraper_types']) + ')' if elements['scraper_types'] else ''} 
                    {' (Errors: ' + str(error_count) + ')' if error_count > 0 else ''}
                </summary>
                <div style="margin: 15px 0;">
                    <details style="margin: 10px 0; background: white; border-radius: 10px; padding: 10px;">
                        <summary style="font-weight: bold; cursor: pointer; color: #2c5282; font-size: 16px;">
                            📋 URLs in this Domain ({url_count} URLs show/hide)
                        </summary>
                        <div style="margin: 15px 0; max-height: 300px; overflow-y: auto; background: #f8f9fa; padding: 15px; border-radius: 8px;">
                            <ul style="margin: 0; padding-left: 20px; list-style-type: none;">
            ''')
            
            for url in sorted(elements['all_urls']):
                html_content.append(f'''
                <li style="margin: 8px 0; padding: 5px; background: white; border-radius: 5px; border-left: 3px solid #6c757d;">
                    <a href="{url}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px; text-decoration: none;">
                        🔗 {url}
                    </a>
                </li>
                ''')
            
            html_content.append('''
                            </ul>
                        </div>
                    </details>
            ''')
            
            # Show why no indicators were found (errors from all scrapers)
            if elements['errors']:
                html_content.append(f'''
                <details style="margin: 15px 0; background: rgba(220, 53, 69, 0.05); border-radius: 10px; border: 1px solid #dc3545;">
                    <summary style="font-weight: bold; color: #dc3545; cursor: pointer; padding: 15px; font-size: 16px;">
                        ❌ Errors ({len(elements['errors'])}) grouped by Task Type
                    </summary>
                    <div style="padding: 0 15px 15px 15px;">
                ''')
                
                # Group errors by scraper type
                errors_by_scraper = defaultdict(list)
                for error in elements['errors']:
                    errors_by_scraper[error['scraper']].append(error)
                
                for scraper, scraper_errors in errors_by_scraper.items():
                    html_content.append(f'''
                    <div style="margin: 15px 0; border-bottom: 1px dashed #dc3545; padding-bottom: 10px;">
                        <h5 style="color: #dc3545; margin: 0 0 10px 0;">🔍 {scraper} Errors ({len(scraper_errors)})</h5>
                    ''')
                    
                    for error in scraper_errors:
                        html_content.append(f'''
                        <div style="border-left: 3px solid #dc3545; padding-left: 15px; margin: 12px 0; background: white; border-radius: 8px; padding: 12px;">
                            <strong>URL:</strong> <a href="{error['url']}" target="_blank" class="clickable-url" style="word-break: break-all; font-size: 14px;">{error['url']}</a><br>
                            <strong>Error:</strong> <span style="color: #dc3545; font-weight: bold;">
                                {", ".join(error['error_messages']) if isinstance(error['error_messages'], list) else error['error_messages']}
                            </span>
                        </div>
                        ''')
                    
                    html_content.append('</div>')
                
                html_content.append('</div></details>')
            else:
                html_content.append(f'''
                <div style="margin: 15px 0; padding: 15px; background: rgba(108, 117, 125, 0.05); border-radius: 10px; border: 1px solid #6c757d;">
                    <p style="margin: 0; color: #6c757d; font-style: italic;">
                        ℹ️ No errors logged. Domain was successfully scanned by all scrapers, but no Passkey indicators found.
                    </p>
                </div>
                ''')
            
            html_content.append('</div></details>')
    
    # Add summary at the top with corrected values - KORRIGIERT
    summary_html = f'''
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 15px; margin: 20px 0; box-shadow: 0 10px 30px rgba(0,0,0,0.2);">
        <h3 style="margin: 0 0 20px 0; font-size: 24px;">📊 Summary</h3>
        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px;">
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 18px; font-weight: bold;">{total_domains}</div>
                <div style="opacity: 0.9;">Total Domains</div>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 18px; font-weight: bold;">{domains_with_elements}</div>
                <div style="opacity: 0.9;">Domains with Indicators</div>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 18px; font-weight: bold;">{domains_without_elements}</div>
                <div style="opacity: 0.9;">Domains without Indicators</div>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 18px; font-weight: bold;">{total_secure_indicators}</div>
                <div style="opacity: 0.9;">Secure Indicators</div>
            </div>
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
                <div style="font-size: 18px; font-weight: bold;">{total_possible_indicators}</div>
                <div style="opacity: 0.9;">Possible Indicators</div>
            </div>
        </div>
    </div>
    '''
    
    return summary_html + ''.join(html_content)

# ====================== MAIN EXECUTION ======================

try:
    # Connect to MongoDB
    print("🔄 Starting MongoDB data extraction...")
    client = pymongo.MongoClient(f'mongodb://{MONGO_USER}:{MONGO_PASS}@{MONGO_HOST}:{MONGO_PORT}/')
    db = client[MONGO_DB_NAME]
    
    print("📡 Loading data from all collections...")
    domain_data = load_all_data(db)
    
    print(f"\n✅ Data extraction complete! Found {len(domain_data)} unique domains.")
    
    print("🔄 Starting enhanced analysis...")
    
    # Count domains with and without elements
    domains_with_elements = 0
    domains_without_elements = 0
    
    for domain, data in domain_data.items():
        elements = analyze_domain_elements(data)
        has_indicators = len(elements['secure_indicators']) > 0 or len(elements['possible_indicators']) > 0
        
        if has_indicators:
            domains_with_elements += 1
        else:
            domains_without_elements += 1
    
    print(f"✅ Analysis complete!")
    print(f"   - Total domains: {len(domain_data)}")
    print(f"   - Domains with indicators: {domains_with_elements}")
    print(f"   - Domains without indicators: {domains_without_elements}")
    print(f"   - Task names analyzed: {', '.join(TASK_NAMES_TO_ANALYZE) if TASK_NAMES_TO_ANALYZE else 'ALL'}")
    
    print("🎨 Creating HTML representation...")
    html_output = create_domain_analysis_html(domain_data)
    
    print("✅ HTML representation generated.")
    print("🔎 Creating detailed analysis of all found elements...")
    
    # Display the HTML
    display(HTML(html_output))
    
    print(f"✅ Detailed analysis complete!")
    print(f"   🌐 Total domains processed: {len(domain_data)}")
    print(f"   🌐 Domains with elements: {domains_with_elements}")
    print(f"   🌐 Domains without elements: {domains_without_elements}")
    print("\n✅ Domain-based element analysis complete! Each domain is analyzed once with all its URLs and findings from all scrapers.")
    print("💡 URL lists are now expandable - click on 'URLs in this Domain' to show/hide the full list.")
    print("🔍 If not all domains are visible, check the search filter - clear the search box to show all domains.")

except Exception as e:
    print(f"❌ Error: {e}")
    print("Please check if MongoDB is running and accessible.")

In [ ]:
# Save Found Elements Results - HTML Export
import os
from datetime import datetime

# Erstelle Output-Ordner
output_dir = r"elements_found"
os.makedirs(output_dir, exist_ok=True)

# Timestamp für eindeutige Dateinamen
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

try:
    # 1. Speichere HTML-Output aus der ersten Zelle
    if 'html_output' in locals() and html_output:
        html_path = os.path.join(output_dir, f"found_elements_analysis_{timestamp}.html")
        
        # Vollständige HTML-Seite erstellen
        complete_html = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Found Elements Analysis - {timestamp}</title>
    <style>
        body {{
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', 'Roboto', sans-serif;
            line-height: 1.6;
            color: #333;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f8f9fa;
        }}
        .container {{
            background: white;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }}
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 30px;
            text-align: center;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🔍 Found Elements Analysis Report</h1>
            <p>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            <p>Collections analyzed: {', '.join(COLLECTIONS_TO_ANALYZE)}</p>
            <p>Tasks analyzed: {', '.join(TASK_NAMES_TO_ANALYZE) if TASK_NAMES_TO_ANALYZE else 'ALL'}</p>
        </div>
        
        {html_output}
        
        <div style="margin-top: 40px; padding: 20px; background: #e9ecef; border-radius: 10px; text-align: center;">
            <p style="margin: 0; color: #6c757d; font-style: italic;">
                📊 Report generated by Found Elements Analysis Notebook<br>
                🕒 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            </p>
        </div>
    </div>
</body>
</html>"""
        
        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(complete_html)
        
        print(f"✅ HTML-Report gespeichert: {os.path.basename(html_path)}")
        print(f"🌐 Öffne in Browser: file:///{html_path.replace(os.sep, '/')}")
    else:
        print("⚠️ Keine HTML-Daten zum Speichern gefunden. Führe zuerst die erste Zelle aus.")

    # 2. Zusammenfassung der Domain-Statistiken als JSON
    if 'domain_data' in locals() and domain_data:
        import json
        
        # Erstelle Domain-Zusammenfassung
        domain_summary = {}
        for domain, data in domain_data.items():
            elements = analyze_domain_elements(data)
            domain_summary[domain] = {
                'url_count': len(elements['all_urls']),
                'secure_indicators': len(elements['secure_indicators']),
                'possible_indicators': len(elements['possible_indicators']),
                'errors': len(elements['errors']),
                'scrapers_used': list(elements['scraper_types']),
                'has_secure_network': elements['has_secure_network'],
                'has_secure_api': elements['has_secure_api'],
                'has_secure_term': elements['has_secure_term']
            }
        
        json_path = os.path.join(output_dir, f"found_elements_domain_summary_{timestamp}.json")
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(domain_summary, f, indent=2, ensure_ascii=False)
        
        print(f"📋 Domain-Zusammenfassung gespeichert: {os.path.basename(json_path)}")

    # 3. Detaillierte Element-Liste als JSON
    if 'domain_data' in locals() and domain_data:
        all_elements = {
            'secure_indicators': [],
            'possible_indicators': [],
            'errors': []
        }
        
        for domain, data in domain_data.items():
            elements = analyze_domain_elements(data)
            all_elements['secure_indicators'].extend(elements['secure_indicators'])
            all_elements['possible_indicators'].extend(elements['possible_indicators'])
            all_elements['errors'].extend(elements['errors'])
        
        elements_json_path = os.path.join(output_dir, f"found_elements_detailed_{timestamp}.json")
        with open(elements_json_path, 'w', encoding='utf-8') as f:
            json.dump(all_elements, f, indent=2, ensure_ascii=False)
        
        print(f"📊 Detaillierte Element-Liste gespeichert: {os.path.basename(elements_json_path)}")

    print(f"\n✅ Alle Dateien gespeichert in: {output_dir}")
    print("📁 Gespeicherte Dateien:")
    for file in sorted(os.listdir(output_dir)):
        if timestamp in file:
            file_path = os.path.join(output_dir, file)
            file_size = os.path.getsize(file_path) / 1024  # KB
            print(f"   📄 {file} ({file_size:.1f} KB)")

except Exception as e:
    print(f"❌ Fehler beim Speichern: {e}")
    print("💡 Stelle sicher, dass die erste Zelle erfolgreich ausgeführt wurde.")